# SciTeX MCP Servers - Code Translation

This notebook demonstrates how to use SciTeX's MCP (Model Context Protocol) servers for translating between SciTeX format and pure Python code.

## 1. Understanding MCP Servers

MCP servers enable AI assistants like Claude to translate code between different formats. SciTeX provides servers for:
- Converting SciTeX code to pure Python
- Converting pure Python to SciTeX format
- Preserving functionality while changing syntax

## 2. Setup and Import

In [ ]:
import scitex
from pathlib import Path
import json

## 3. Example: SciTeX to Pure Python Translation

Here's a typical SciTeX script that we want to translate:

In [ ]:
# Example SciTeX code
scitex_code = '''
import scitex
import numpy as np

# Load data using SciTeX
data = scitex.io.load("data.npz")
X = data['X']
y = data['y']

# Create figure with SciTeX
fig, ax = scitex.plt.subplots(figsize=(8, 6))
ax.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_title('Data Distribution')
fig.save('scatter_plot.png')

# Save processed data
processed = {
    'X_normalized': (X - X.mean()) / X.std(),
    'metadata': {'original_shape': X.shape}
}
scitex.io.save(processed, 'processed_data.pkl')
'''

# Save example SciTeX code
example_dir = Path("./mcp_examples")
example_dir.mkdir(exist_ok=True)
scitex_file = example_dir / "example_scitex.py"
scitex.io.save(scitex_code, scitex_file)
print(f"Saved SciTeX example to {scitex_file}")

## 4. Pure Python Equivalent

The MCP server would translate the above to pure Python:

In [ ]:
# Equivalent pure Python code (what MCP server produces)
pure_python_code = '''
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Load data using numpy
data = np.load("data.npz")
X = data['X']
y = data['y']

# Create figure with matplotlib
fig, ax = plt.subplots(figsize=(8, 6))
scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_title('Data Distribution')
plt.tight_layout()
fig.savefig('scatter_plot.png', dpi=300, bbox_inches='tight')
plt.close()

# Save processed data
processed = {
    'X_normalized': (X - X.mean()) / X.std(),
    'metadata': {'original_shape': X.shape}
}
with open('processed_data.pkl', 'wb') as f:
    pickle.dump(processed, f)
'''

# Save pure Python example
pure_python_file = example_dir / "example_pure_python.py"
scitex.io.save(pure_python_code, pure_python_file)
print(f"Saved pure Python example to {pure_python_file}")

## 5. MCP Server Configuration

MCP servers are configured in Claude's settings. Here's an example configuration:

In [ ]:
# Example MCP server configuration
mcp_config = {
    "mcpServers": {
        "scitex-translator": {
            "command": "python",
            "args": ["-m", "scitex.mcp_servers.translator"],
            "env": {
                "PYTHONPATH": str(Path.cwd())
            }
        }
    }
}

# Save configuration example
config_file = example_dir / "mcp_config_example.json"
scitex.io.save(mcp_config, config_file)
print(f"\nMCP configuration saved to {config_file}")
print(json.dumps(mcp_config, indent=2))

## 6. Translation Patterns

Common translation patterns handled by MCP servers:

In [ ]:
# Create a translation mapping reference
translation_patterns = {
    "I/O Operations": [
        {"scitex": "scitex.io.save(data, 'file.pkl')",
         "python": "with open('file.pkl', 'wb') as f:\n    pickle.dump(data, f)"},
        {"scitex": "data = scitex.io.load('file.json')",
         "python": "with open('file.json', 'r') as f:\n    data = json.load(f)"},
        {"scitex": "df = scitex.io.load('data.csv')",
         "python": "df = pd.read_csv('data.csv')"}
    ],
    "Plotting": [
        {"scitex": "fig, ax = scitex.plt.subplots()",
         "python": "fig, ax = plt.subplots()"},
        {"scitex": "fig.save('plot.png')",
         "python": "fig.savefig('plot.png', dpi=300, bbox_inches='tight')"}
    ],
    "Decorators": [
        {"scitex": "@scitex.decorators.cache_disk",
         "python": "# Implement custom disk caching or use joblib"},
        {"scitex": "@scitex.decorators.time",
         "python": "# Use time.time() or timeit module"}
    ]
}

# Display translation patterns
for category, patterns in translation_patterns.items():
    print(f"\n{category}:")
    for pattern in patterns:
        print(f"  SciTeX: {pattern['scitex']}")
        print(f"  Python: {pattern['python']}")
        print()

## 7. Practical Example: Data Processing Pipeline

In [ ]:
# SciTeX version of a data processing pipeline
scitex_pipeline = '''
import scitex
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

@scitex.decorators.time
@scitex.decorators.cache_disk
def process_data(input_file):
    """Process data with caching and timing."""
    # Load data
    data = scitex.io.load(input_file)
    X = data['features']
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    # Visualize
    fig, ax = scitex.plt.subplots(figsize=(8, 6))
    ax.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.6)
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
    ax.set_title('PCA Results')
    
    return {
        'X_pca': X_pca,
        'scaler': scaler,
        'pca': pca,
        'figure': fig
    }

# Run pipeline
results = process_data('raw_data.npz')
results['figure'].save('pca_results.png')
scitex.io.save(results, 'processed_results.pkl')
'''

# Save pipeline example
pipeline_file = example_dir / "pipeline_scitex.py"
scitex.io.save(scitex_pipeline, pipeline_file)
print(f"Saved SciTeX pipeline to {pipeline_file}")

## 8. Benefits of MCP Translation

1. **Portability**: Convert SciTeX code to run in environments without SciTeX
2. **Learning**: Understand how SciTeX features map to standard Python
3. **Integration**: Mix SciTeX with pure Python codebases
4. **Flexibility**: Choose the right tool for each task

In [ ]:
# Example: Feature comparison
feature_comparison = pd.DataFrame({
    'Feature': ['Unified I/O', 'Smart Plotting', 'Caching', 'Type Conversion', 'Error Handling'],
    'SciTeX': ['✓ One interface', '✓ Auto-configuration', '✓ Built-in decorators', 
               '✓ Automatic', '✓ Comprehensive'],
    'Pure Python': ['Multiple libraries', 'Manual setup', 'Custom implementation', 
                   'Manual handling', 'Basic try/except']
})

print("\nFeature Comparison:")
print(feature_comparison.to_string(index=False))

# Save comparison
scitex.io.save(feature_comparison, example_dir / 'feature_comparison.csv')

## 9. Using MCP Servers with Claude

To use MCP servers with Claude:

1. Configure the MCP server in Claude's settings
2. Ask Claude to translate your code
3. Claude will use the MCP server automatically

Example prompts:
- "Convert this SciTeX script to pure Python"
- "Translate this matplotlib code to use SciTeX"
- "Show me both SciTeX and pure Python versions"

## 10. Advanced Translation Example

In [ ]:
# Complex example with multiple SciTeX features
advanced_example = '''
import scitex
import numpy as np
from scitex.decorators import time, cache_disk, numpy_fn

@numpy_fn
@cache_disk
def analyze_signals(signals, sampling_rate=1000):
    """Analyze multiple signals with SciTeX features."""
    # Ensure 3D format
    signals = scitex.dsp.ensure_3d(signals)
    
    # Filter signals
    filtered = scitex.dsp.filt.bandpass(signals, low=1, high=50, fs=sampling_rate)
    
    # Compute power spectral density
    psd = scitex.dsp.psd(filtered, fs=sampling_rate)
    
    # Create multi-panel figure
    fig, axes = scitex.plt.subplots(2, 2, figsize=(12, 8))
    
    # Plot raw signal
    axes[0, 0].plot(signals[0, 0, :1000])
    axes[0, 0].set_title('Raw Signal')
    
    # Plot filtered signal
    axes[0, 1].plot(filtered[0, 0, :1000])
    axes[0, 1].set_title('Filtered Signal')
    
    # Plot PSD
    axes[1, 0].semilogy(psd['freqs'], psd['psd'][0, 0])
    axes[1, 0].set_xlabel('Frequency (Hz)')
    axes[1, 0].set_ylabel('Power')
    axes[1, 0].set_title('Power Spectral Density')
    
    # Statistics
    stats = scitex.stats.describe(filtered)
    axes[1, 1].text(0.1, 0.5, str(stats), transform=axes[1, 1].transAxes)
    axes[1, 1].axis('off')
    axes[1, 1].set_title('Statistics')
    
    fig.tight_layout()
    
    return {
        'filtered': filtered,
        'psd': psd,
        'stats': stats,
        'figure': fig
    }
'''

# This demonstrates features that would require significant pure Python code:
# - Automatic numpy conversion (@numpy_fn)
# - Disk caching with smart invalidation
# - DSP functions with consistent API
# - Enhanced plotting with automatic configuration
# - Unified statistics interface

print("Advanced example demonstrates SciTeX advantages:")
print("- Decorators for automatic optimization")
print("- Unified DSP interface")
print("- Smart plotting defaults")
print("- Consistent API across domains")

## Summary

This notebook demonstrated:
- Understanding MCP servers for code translation
- Examples of SciTeX to pure Python conversion
- Configuration of MCP servers
- Common translation patterns
- Benefits of using MCP translation
- Integration with Claude for automatic translation

MCP servers bridge the gap between SciTeX's convenience and pure Python's portability, giving you the best of both worlds.